### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT10UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

In [13]:
# -------
# IMPORT
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import BSMModel, SABRModel, HestonHullWhiteModel

pd.options.display.max_rows = 30
pd.options.display.max_columns = 30
pd.options.display.float_format = '{:,.4f}'.format

In [107]:
def compute_voltarget_price(
        F0, S0, K,
        v0, r0, b0,
        kappa_v, theta_v, sigma_v,
        kappa_r, theta_r, sigma_r,
        rho_sv, rho_sr, rho_vr,
        T, M, N,
        seed, seed_value,
        init_r_var,
        lambda_l, lambda_s,
        target_vol, max_beta,
        lag,
        excess_return,
        get_extra_outputs):
    S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)

    S_df = pd.DataFrame(S)
    r_df = pd.DataFrame(r)

    S_df.iloc[0, :] = F0

    log_returns = np.log(S_df / S_df.shift(1)).pow(2).fillna(init_r_var)

    alpha_l = 1 - lambda_l
    variance_l = log_returns.ewm(alpha=alpha_l, adjust=False).mean()

    alpha_s = 1 - lambda_s
    variance_s = log_returns.ewm(alpha=alpha_s, adjust=False).mean()

    r_vol_l = np.sqrt(variance_l * 252)
    r_vol_s = np.sqrt(variance_s * 252)

    realized_vol = r_vol_l.combine(r_vol_s, np.maximum)

    beta = (target_vol / realized_vol.shift(lag)).clip(upper=max_beta).bfill()

    vt_returns = (S_df.pct_change().fillna(0) * beta)
    if excess_return:
        risk_free = beta * (r_df * (T / M))
        vt_returns -= risk_free

    vt_price = vt_returns.add(1).cumprod() * 100
    vt_call_price = (vt_price.iloc[-1].add(-K).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()

    if get_extra_outputs:
        return {'call_price': vt_call_price,
                'average_beta': beta.mean().mean(),
                'vt_forrward': vt_price.iloc[-1].mean(),
                'average_realized_vol': np.sqrt(vt_returns.pow(2).mean().mean() * 252)}
    else:
        return vt_call_price

In [112]:
def compute_numeric_derivative(pricing_fun, base_params, param_names, h=None):
    """
    Numerically compute derivative(s) of pricing_fun at base_params.

    - pricing_fun: callable(**kwargs) -> scalar price
    - base_params: dict of parameters to pass to pricing_fun
    - param_names: str or (str, str)
        * If str -> first derivative w.r.t that parameter (central difference)
        * If (p, p) -> second derivative w.r.t p (second central difference)
        * If (p, q) with p != q -> mixed second partial d2/dp dq (central)
    - h: optional bump size (scalar or dict mapping param->h). If None, automatic h used.

    Returns numeric derivative (float).
    """
    import numpy as np

    def safe_eval(params):
        val = pricing_fun(**params)
        a = np.asarray(val)
        if a.size != 1:
            raise ValueError("pricing_fun must return a scalar-like value")
        return float(a.item())

    eps = np.finfo(float).eps
    # normalize param_names to tuple
    if isinstance(param_names, str):
        params = (param_names,)
    else:
        params = tuple(param_names)

    # helper to compute adaptive h for a single param
    def get_h_for(name, x):
        if h is None:
            return (eps ** (1/3)) * (abs(x) + 1.0)
        if isinstance(h, dict):
            return float(h.get(name, (eps ** (1/3)) * (abs(x) + 1.0)))
        return float(h)

    # single derivative (first)
    if len(params) == 1:
        p = params[0]
        x0 = float(base_params[p])
        hh = get_h_for(p, x0)
        p_plus = dict(base_params); p_plus[p] = x0 + hh
        p_minus = dict(base_params); p_minus[p] = x0 - hh
        f_plus = safe_eval(p_plus)
        f_minus = safe_eval(p_minus)
        return (f_plus - f_minus) / (2.0 * hh)

    # two params -> second derivative or mixed partial
    if len(params) == 2:
        p, q = params
        x_p = float(base_params[p])
        x_q = float(base_params[q])
        h_p = get_h_for(p, x_p)
        h_q = get_h_for(q, x_q)

        if p == q:
            # second derivative w.r.t same parameter
            p_plus = dict(base_params); p_plus[p] = x_p + h_p
            p_minus = dict(base_params); p_minus[p] = x_p - h_p
            f_plus = safe_eval(p_plus)
            f0 = safe_eval(base_params)
            f_minus = safe_eval(p_minus)
            return (f_plus - 2.0 * f0 + f_minus) / (h_p ** 2)
        else:
            # mixed partial: central 4-point formula
            pp = dict(base_params); pp[p] = x_p + h_p; pp[q] = x_q + h_q
            pm = dict(base_params); pm[p] = x_p + h_p; pm[q] = x_q - h_q
            mp = dict(base_params); mp[p] = x_p - h_p; mp[q] = x_q + h_q
            mm = dict(base_params); mm[p] = x_p - h_p; mm[q] = x_q - h_q
            f_pp = safe_eval(pp)
            f_pm = safe_eval(pm)
            f_mp = safe_eval(mp)
            f_mm = safe_eval(mm)
            return (f_pp - f_pm - f_mp + f_mm) / (4.0 * h_p * h_q)

    raise ValueError("param_names must be a string or a tuple/list of two strings")

In [ ]:
# Default Parameters
params = {
    # Initial values
    "F0": 100,                   # Initial Fixing
    "K": 100,                    # Strike
    "S0": 100.0,                 # Initial stock price
    "v0": 0.17**2,               # Initial variance
    "r0": 0.036,                 # Initial short rate
    "b0": 0.0033,                # Initial gross spread

    # Variance process parameters
    "kappa_v": 2.5,              # Mean reversion speed for variance
    "theta_v": 0.17**2,           # Long-term variance
    "sigma_v": 0.35,              # Volatility of variance

    # Interest rate process parameters
    "kappa_r": 1.0,             # Mean reversion speed for rates
    "theta_r": 0.025,             # Long-term interest rate
    "sigma_r": 0.015,          # Volatility of interest rate

    # Correlation parameters
    "rho_sv": -0.6,              # Stock-variance correlation
    "rho_sr": 0.2,               # Stock-rate correlation
    "rho_vr": -0.2,              # Variance-rate correlation

    # Simulation parameters
    "T": 1.0,                    # Time horizon in years
    "M": 252,                    # Number of simulations
    "N": 10000,                  # Number of time steps
    "seed": True,                # Use seed for reproducibility
    "seed_value": 44,            # Seed value

    "init_r_var": (0.17**2) / 252,

    # Risk control parameters
    "lambda_l": 0.97,
    "lambda_s": 0.94,
    "target_vol": 0.1,
    "max_beta": 1.5,
    "lag": 2,

    "excess_return": True,

    'get_extra_outputs': False,
}

In [153]:
delta = compute_numeric_derivative(compute_voltarget_price, params, 'S0')
gamma = compute_numeric_derivative(compute_voltarget_price, params, ('S0','S0'))
theta = compute_numeric_derivative(compute_voltarget_price, params, ('T'))
print('delta', delta)
print('gamma', gamma)
print('theta', theta/-250)

delta 0.31089663440690846
gamma -0.017112437658429167
theta -0.0009032922045717681


In [155]:
params_ = {**params}
params_['get_extra_outputs'] = True
compute_voltarget_price(**params_)

{'call_price': 3.9744069397599087,
 'average_beta': 0.6765143202676414,
 'vt_forrward': 100.38080382175526,
 'average_realized_vol': 0.09696662372964951}

In [161]:
params_ = {**params}
params_['v0'] = 0.2**2
params_['get_extra_outputs'] = True
compute_voltarget_price(**params_)

{'call_price': 3.9833370963150774,
 'average_beta': 0.6343657660947094,
 'vt_forrward': 100.37436596208128,
 'average_realized_vol': 0.09729273574808975}

In [159]:
params_ = {**params}
params_['v0'] = 0.2**2
params_['theta_v'] = 0.2**2
params_['get_extra_outputs'] = True
compute_voltarget_price(**params_)

{'call_price': 4.063077832002311,
 'average_beta': 0.566025845862799,
 'vt_forrward': 100.35644571884563,
 'average_realized_vol': 0.09959483181129497}

In [167]:
params_ = {**params}
params_['init_r_var'] = (0.1**2) / 252
params_['get_extra_outputs'] = True
compute_voltarget_price(**params_)

{'call_price': 4.220322827360221,
 'average_beta': 0.7142957004357975,
 'vt_forrward': 100.41300938197651,
 'average_realized_vol': 0.10272849165363304}